In [1]:
import psycopg2

In [2]:
# connect to the database
try:
    conn = psycopg2.connect(database="postgres",
                            user="postgres",
                            password="apassword",
                            host="localhost")
    conn.autocommit = True
    cur = conn.cursor()
    print("Successfully connected to the database")

except:
    print("Cannot connect to database")

Successfully connected to the database


In [3]:
# load the id and phones from the ad data
cur.execute("""SELECT id, phone
                FROM backpage;""")

data = [record for record in cur]

In [5]:
# create a table to hold the unique phones
cur.execute("""CREATE TABLE IF NOT EXISTS backpage_phone ( 
               id SERIAL PRIMARY KEY NOT NULL, 
               phone VARCHAR UNIQUE NOT NULL);""")

In [6]:
# load the phones into the phone table
for i in data:
    
    # loop over the phones found in each ad...
    phones = i[1].split(';')
    for p in phones:
    
        # if the phone number is not blank...
        if p != '':
            
            # try to load the phone number into the table
            try:

                # add to the phone table
                cur.execute("""INSERT INTO backpage_phone
                               (phone) 
                                VALUES (%s);""", [p])
        
            # if it won't load, it might already be in there, so move on
            except:
                pass

In [8]:
# create a table to hold the phone_to_ad relationships
cur.execute("""CREATE TABLE IF NOT EXISTS backpage_adphone(
               PRIMARY KEY(phone_id, ad_id),
               phone_id integer NOT NULL,
               ad_id integer NOT NULL);""")

In [9]:
# get the phone numbers and ids from the phone table
cur.execute("""SELECT id, phone
                FROM backpage_phone""")

# store the query results in a dict (phone=key, id=value)
all_phones = {}
for record in cur:
    all_phones[record[1]] = record[0]

In [10]:
len(all_phones)

1503

In [11]:
all_phones

{'5044467345': 4329,
 '5048107688': 246,
 '4242751929': 2690,
 '2055585064': 4815,
 '3188169506': 859,
 '9106010150': 632,
 '2817471121': 91,
 '3202875433': 3327,
 '9804042612': 807,
 '4237551658': 1283,
 '6787346920': 1451,
 '3375415264': 2313,
 '7027413738': 1335,
 '2252583231': 3142,
 '6467409328': 645,
 '5046033811': 4002,
 '5045004071': 154,
 '3525307862': 1175,
 '22532579912': 3845,
 '2254143109': 1497,
 '2513874210': 3502,
 '4697359178': 2505,
 '9016713444': 46,
 '3183818648': 4389,
 '15048144187': 2290,
 '3187759942': 3340,
 '9045525343': 675,
 '9729469243': 1648,
 '3183891216': 2703,
 '2252715767': 618,
 '3185201090': 1611,
 '2259009564': 3138,
 '2258035325': 655,
 '8326505310': 1251,
 '2252869925': 2927,
 '9016129537': 3725,
 '243453103': 510,
 '5394249718': 345,
 '3372106475': 410,
 '9292291936': 36,
 '9293004213': 2636,
 '2816902231': 1213,
 '2812458376': 374,
 '8504053415': 718,
 '2252242213': 683,
 '5042769033': 3379,
 '2254560215': 1528,
 '4693784467': 1644,
 '5036068108

In [12]:
data[:5]

[(1, '3187046000'),
 (2, ''),
 (3, '3183218820'),
 (4, '3372571723'),
 (5, '3376547053')]

In [13]:
# load the phone id and ad id into the phone_to_ad table
for i in data:
    
    # loop over the phones found in each ad...
    phones = i[1].split(';')
    for p in phones:
    
        # if the phone number is not blank...
        if p != '':
            
            phone_id = all_phones[p]
            ad_id = i[0]
            #phone_ad = "phone:" + str(phone_id) + "-" + "ad:" + str(ad_id)
            
            # try to load the phone number into the table
            try:

                # add to the phone table
                cur.execute("""INSERT INTO backpage_adphone
                               (phone_id, ad_id) 
                                VALUES (%s, %s);""", [phone_id, ad_id])
        
            # if it won't load, it might already be in there, so move on
            except:
                pass